In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# Create a dataset, a dataloader

In [2]:
# Create user tower and item tower

In [3]:
# Create two-towers model

In [ ]:
# Train / Test split

In [4]:
# Create a in-batch negative dataset that generates negative training samples from other user's positive items
# The negative sampling dataset needs to be created for training and test individually

In [5]:
# Train the two-towers model and monitor the loss and training epoch
# * Use Binary Cross Entropy loss
# * Switch to Negative Contrastive Ex loss

In [6]:
# Calculate offline performance metrics on the train and test set
# Hit@K
# Recall@K
# Normalised Weighted Cumulative Gain@K
# Mean Reciprocal Rank@K